# Keystone Project - Algorithmic Trading Using a Contrarian Strategy (That didn't work well :( I should try a momentum strategy next!

__Assignment / Goal:__

Create a Trading Algorithm for a Contrarian Trading Strategy: 

1. At the end of each trading day,  __buy__ the __three worst performing stocks__ among the __30 DJIA__ stocks and hold them for the next trading day.

2. At the end of each trading day,  __short sell__ the __three best performing stocks__ among the __30 DJIA__ stocks and short them for the next trading day.

3. We buy/sell __one share__ of the respective stocks.

4. Underlying idea: The __Market overreacted__ and most recent (extreme) price __trends will revert__ in the near future -> __Contrarian Trading__ (!= Momentum Trading)

5. This code does the following:
- __imports__ required data 
- __measures__ the most recent performance (price change in %)
- __identifies__ the best/worst performing stocks
- __trades__ the stocks with the IBKR API 

In [155]:
import pandas as pd
import yfinance as yf
from ib_insync import *
util.startLoop()

## Get the Symbols

In [38]:
df = pd.read_csv("DJI_Const.csv", header = [0, 1], index_col = 0, parse_dates = [0])
df

Adj Close                                                  \
                  AAPL        AMGN         AXP          BA         CAT   
Date                                                                     
2020-08-31  127.289726  238.217224   98.721237  171.820007  135.369568   
2020-09-01  132.360016  235.903885   99.576393  172.100006  138.927170   
2020-09-02  129.617706  242.731033  101.451889  174.779999  141.990143   
2020-09-03  119.240417  233.129745  101.510201  168.770004  139.602539   
2020-09-04  119.319313  233.590546  102.686020  171.050003  140.953293   
...                ...         ...         ...         ...         ...   
2022-11-09  134.869995  289.649994  145.860001  168.740005  225.539993   
2022-11-10  146.869995  291.010010  154.750000  177.580002  232.449997   
2022-11-11  149.699997  285.019989  154.889999  177.490005  236.490005   
2022-11-14  148.279999  285.299988  154.130005  173.550003  236.500000   
2022-11-15  150.039993  283.600006  153.889999  175.350006  234.589996   

                                                                      ...  \
                   CRM       CSCO         CVX         DIS        DOW  ...   
Date                                                                  ...   
2020-08-31  272.649994  39.405495   76.447708  131.869995  41.147762  ...   
2020-09-01  281.250000  39.228161   75.673492  133.550003  43.108482  ...   
2020-09-02  276.690002  39.592159   75.773689  135.389999  44.886814  ...   
2020-09-03  265.010010  38.220158   74.944817  133.240005  44.129879  ...   
2020-09-04  254.699997  38.098816   74.626007  131.990005  44.321396  ...   
...                ...        ...         ...         ...        ...  ...   
2022-11-09  142.059998  43.910000  177.929993   86.750000  47.680000  ...   
2022-11-10  156.300003  45.660000  181.300003   90.459999  49.959999  ...   
2022-11-11  157.729996  44.790001  186.460007   95.010002  53.139999  ...   
2022-11-14  158.660004  44.740002  186.550003   94.279999  51.950001  ...   
2022-11-15  162.070007  44.900002  188.050003   95.510002  51.599998  ...   

              Volume                                                           \
                MSFT       NKE       PG      TRV      UNH         V        VZ   
Date                                                                            
2020-08-31  28774200   3519800  5596800  2297900  4341600   9326900  14555500   
2020-09-01  25725500   4912400  5671800   969900  2487100   5642900  13358400   
2020-09-02  34080800   6775400  8384800  1391400  2846500   9863800  21711900   
2020-09-03  58400300   7264400  7277900  1299400  3872500  11310200  23126800   
2020-09-04  59664100   5157600  7437700  1460700  2901000  13243600  22250100   
...              ...       ...      ...      ...      ...       ...       ...   
2022-11-09  27852900   6550900  5690700   999200  2672600   7320200  17574600   
2022-11-10  46268000   9361000  8068700  1618000  3426400   8048900  23049000   
2022-11-11  34600900  12595600  7174400  1403300  7032900   9600700  22978900   
2022-11-14  31123300   7134900  8516200  1427300  5238400   5796100  23028500   
2022-11-15  31367300   7241000  7731700  1445900  5102800   7315700  25058900   

                                           
                 WBA       WMT       ^DJI  
Date                                       
2020-08-31   7940100  15078800  517320000  
2020-09-01  10728300  35599400  423410000  
2020-09-02   7672400  17222000  539510000  
2020-09-03   8278000  16005000  650080000  
2020-09-04   6994400  11327400  694640000  
...              ...       ...        ...  
2022-11-09   6830500   5310700  355200000  
2022-11-10   8825000   6770200  480130000  
2022-11-11  13098600   6419700  427790000  
2022-11-14   8301900  12508600  354340000  
2022-11-15   6755300  25504700  390610000  

[558 rows x 186 columns]

In [41]:
symbols = df.Close.columns.to_list()
symbols

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT',
 '^DJI']

In [43]:
symbols.remove("^DJI")

In [45]:
len(symbols)

30

## Get most recent Prices and Performance

In [56]:
yf.Ticker(ticker = "AAPL").get_fast_info() 

lazy-loading dict with keys = ['currency', 'dayHigh', 'dayLow', 'exchange', 'fiftyDayAverage', 'lastPrice', 'lastVolume', 'marketCap', 'open', 'previousClose', 'quoteType', 'regularMarketPreviousClose', 'shares', 'tenDayAverageVolume', 'threeMonthAverageVolume', 'timezone', 'twoHundredDayAverage', 'yearChange', 'yearHigh', 'yearLow']

In [57]:
cprice =  yf.Ticker("AAPL").get_fast_info()["last_price"]
cprice

226.64999389648438

In [58]:
last_close = yf.Ticker("AAPL").get_fast_info()["regularMarketPreviousClose"] 
last_close

233.0

In [59]:
perf = cprice / last_close - 1
perf

-0.02725324507946625

In [60]:
perf = pd.Series(dtype = float)
perf

Series([], dtype: float64)

In [61]:
symbols

['AAPL',
 'AMGN',
 'AXP',
 'BA',
 'CAT',
 'CRM',
 'CSCO',
 'CVX',
 'DIS',
 'DOW',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'JPM',
 'KO',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'V',
 'VZ',
 'WBA',
 'WMT']

In [62]:
count = 1
for symbol in symbols:
    try:
        fast_info = yf.Ticker(ticker = symbol).get_fast_info() 
        prc_chg = fast_info["last_price"] / fast_info["regularMarketPreviousClose"] - 1 
        perf.loc[symbol] = prc_chg
        print(count, end = '\r')
        count += 1
    except Exception as e:
        print("{} not found".format(symbol))
print("Download complete.")

Download complete.


In [64]:
perf

AAPL   -0.027253
AMGN   -0.001893
AXP    -0.005052
BA     -0.014273
CAT    -0.002339
CRM    -0.001534
CSCO   -0.008268
CVX     0.002446
DIS    -0.009605
DOW    -0.005125
GS     -0.006544
HD     -0.006762
HON    -0.002951
IBM    -0.012529
INTC   -0.007886
JNJ    -0.001975
JPM    -0.015698
KO      0.006262
MCD    -0.002430
MMM    -0.007974
MRK    -0.000704
MSFT   -0.016663
NKE    -0.002602
PG      0.001415
TRV    -0.001324
UNH    -0.001026
V       0.001891
VZ      0.001002
WBA    -0.011719
WMT     0.001434
dtype: float64

In [163]:
perf.sort_values(inplace = True) #sorts according to percentage change to find the worst performing and best performing ones
perf

symbol
KO     -1.000000
V      -1.000000
CVX    -1.000000
BA     -0.014273
IBM    -0.012529
WBA    -0.011719
DIS    -0.009605
CSCO   -0.008268
MMM    -0.007974
INTC   -0.007886
HD     -0.006762
GS     -0.006544
DOW    -0.005125
AXP    -0.005052
HON    -0.002951
NKE    -0.002602
MCD    -0.002430
CAT    -0.002339
JNJ    -0.001975
AMGN   -0.001893
CRM    -0.001534
TRV    -0.001324
UNH    -0.001026
MRK    -0.000704
VZ      0.001002
PG      0.001415
WMT     0.001434
MSFT    1.000000
JPM     1.000000
AAPL    1.000000
dtype: float64

In [68]:
perf.index.name = "symbol"
perf

symbol
AAPL   -0.027253
MSFT   -0.016663
JPM    -0.015698
BA     -0.014273
IBM    -0.012529
WBA    -0.011719
DIS    -0.009605
CSCO   -0.008268
MMM    -0.007974
INTC   -0.007886
HD     -0.006762
GS     -0.006544
DOW    -0.005125
AXP    -0.005052
HON    -0.002951
NKE    -0.002602
MCD    -0.002430
CAT    -0.002339
JNJ    -0.001975
AMGN   -0.001893
CRM    -0.001534
TRV    -0.001324
UNH    -0.001026
MRK    -0.000704
VZ      0.001002
PG      0.001415
WMT     0.001434
V       0.001891
CVX     0.002446
KO      0.006262
dtype: float64

## Determine Target Positions

In [76]:
buy_stocks = 3 # buy the 3 worst performing stocks

In [77]:
sell_stocks = 3 # short sell the 3 best performing stocks

In [78]:
shares = 1 # one share per stock

In [79]:
perf.iloc[:buy_stocks] = shares 
perf.iloc[-sell_stocks:] = -shares
perf

symbol
AAPL    1.000000
MSFT    1.000000
JPM     1.000000
BA     -0.014273
IBM    -0.012529
WBA    -0.011719
DIS    -0.009605
CSCO   -0.008268
MMM    -0.007974
INTC   -0.007886
HD     -0.006762
GS     -0.006544
DOW    -0.005125
AXP    -0.005052
HON    -0.002951
NKE    -0.002602
MCD    -0.002430
CAT    -0.002339
JNJ    -0.001975
AMGN   -0.001893
CRM    -0.001534
TRV    -0.001324
UNH    -0.001026
MRK    -0.000704
VZ      0.001002
PG      0.001415
WMT     0.001434
V      -1.000000
CVX    -1.000000
KO     -1.000000
dtype: float64

In [80]:
target = pd.concat([perf.iloc[:buy_stocks], perf.iloc[-sell_stocks:]]).to_frame().reset_index() #what we need
target.columns = ["symbol", "position"]
target

,symbol,position
0,AAPL,1.0
1,MSFT,1.0
2,JPM,1.0
3,V,-1.0
4,CVX,-1.0
5,KO,-1.0


## Identify Current Positions

In [106]:
ib = IB()

In [108]:
ib.connect() #connects to interactive brokers api

<IB connected to 127.0.0.1:7497 clientId=1>

In [110]:
pos = ib.positions()
pos

[Position(account='DUA408233', contract=Stock(conId=265598, symbol='AAPL', exchange='NASDAQ', currency='USD', localSymbol='AAPL', tradingClass='NMS'), position=1.0, avgCost=226.46),
 Position(account='DUA408233', contract=Stock(conId=272093, symbol='MSFT', exchange='NASDAQ', currency='USD', localSymbol='MSFT', tradingClass='NMS'), position=10.0, avgCost=425.069)]

In [112]:
df = util.df(pos)
df

,account,contract,position,avgCost
0,DUA408233,"Stock(conId=265598, symbol='AAPL', exchange='N...",1.0,226.460
1,DUA408233,"Stock(conId=272093, symbol='MSFT', exchange='N...",10.0,425.069


In [114]:
df["symbol"] = df.contract.apply(lambda x: x.symbol)
df["conID"] = df.contract.apply(lambda x: x.conId)
df

,account,contract,position,avgCost,symbol,conID
0,DUA408233,"Stock(conId=265598, symbol='AAPL', exchange='N...",1.0,226.460,AAPL,265598
1,DUA408233,"Stock(conId=272093, symbol='MSFT', exchange='N...",10.0,425.069,MSFT,272093


In [116]:
if df is not None:
    df["symbol"] = df.contract.apply(lambda x: x.symbol)
    df["conID"] = df.contract.apply(lambda x: x.conId)
else: 
    df = pd.DataFrame(columns = ["symbol", "position"])

In [118]:
df

,account,contract,position,avgCost,symbol,conID
0,DUA408233,"Stock(conId=265598, symbol='AAPL', exchange='N...",1.0,226.460,AAPL,265598
1,DUA408233,"Stock(conId=272093, symbol='MSFT', exchange='N...",10.0,425.069,MSFT,272093


## Determine Required Trades (from actual to target positions)

In [122]:
target

,symbol,position
0,AAPL,1.0
1,MSFT,1.0
2,JPM,1.0
3,V,-1.0
4,CVX,-1.0
5,KO,-1.0


In [124]:
df

,account,contract,position,avgCost,symbol,conID
0,DUA408233,"Stock(conId=265598, symbol='AAPL', exchange='N...",1.0,226.460,AAPL,265598
1,DUA408233,"Stock(conId=272093, symbol='MSFT', exchange='N...",10.0,425.069,MSFT,272093


In [126]:
trades = pd.merge(target, df[["symbol", "position"]], "outer", on = "symbol", suffixes = ["_t", "_a"])
trades

,symbol,position_t,position_a
0,AAPL,1.0,1.0
1,CVX,-1.0,NaN
2,JPM,1.0,NaN
3,KO,-1.0,NaN
4,MSFT,1.0,10.0
5,V,-1.0,NaN


In [128]:
trades.fillna(0, inplace = True)
trades

,symbol,position_t,position_a
0,AAPL,1.0,1.0
1,CVX,-1.0,0.0
2,JPM,1.0,0.0
3,KO,-1.0,0.0
4,MSFT,1.0,10.0
5,V,-1.0,0.0


In [130]:
trades["trades"] = trades.position_t - trades.position_a #trades column identifies the position we are to take

In [132]:
trades 

,symbol,position_t,position_a,trades
0,AAPL,1.0,1.0,0.0
1,CVX,-1.0,0.0,-1.0
2,JPM,1.0,0.0,1.0
3,KO,-1.0,0.0,-1.0
4,MSFT,1.0,10.0,-9.0
5,V,-1.0,0.0,-1.0


In [134]:
trades = trades[trades.trades !=0].set_index("symbol").copy() #filters only those where we need an actionable step
trades

,position_t,position_a,trades
symbol,,,
CVX,-1.0,0.0,-1.0
JPM,1.0,0.0,1.0
KO,-1.0,0.0,-1.0
MSFT,1.0,10.0,-9.0
V,-1.0,0.0,-1.0


## Execute Trades

In [139]:
for symbol in trades.index:
    to_trade = trades.loc[symbol, "trades"]
    if to_trade > 0: 
        side = "BUY"
    elif to_trade < 0:
        side = "SELL"
    contract = Stock(symbol, "SMART", "USD")
    cds = ib.reqContractDetails(contract)
    if len(cds) == 0:
        print("No Contract for {} found.".format(symbol))
    elif len(cds) == 1:
        contract = cds[0].contract
        order = MarketOrder(side, abs(to_trade))
        trade = ib.placeOrder(contract, order)
        while not trade.isDone():
            ib.waitOnUpdate()
        if trade.orderStatus.status == "Filled":
            print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
        else:
            print("{} {} failed.".format(side, symbol))
    else:
        contract = cds[0].contract
        print("Multiple Contracts for {} found.".format(symbol))
        order = MarketOrder(side, abs(to_trade))
        trade = ib.placeOrder(contract, order)
        while not trade.isDone():
            ib.waitOnUpdate()
        if trade.orderStatus.status == "Filled":
            print("{} {} @ {}".format(side, symbol, trade.orderStatus.avgFillPrice))
        else:
            print("{} {} failed.".format(side, symbol))  
pos = ib.positions()
df = util.df(pos)
df["symbol"] = df.contract.apply(lambda x: x.symbol)
df["conID"] = df.contract.apply(lambda x: x.conId)
df

SELL CVX @ 146.5
BUY JPM @ 210.09
SELL KO @ 71.94
SELL MSFT @ 428.0
SELL V @ 275.03


,account,contract,position,avgCost,symbol,conID
0,DUA408233,"Stock(conId=265598, symbol='AAPL', exchange='N...",1.0,226.460000,AAPL,265598
1,DUA408233,"Stock(conId=5684, symbol='CVX', exchange='NYSE...",-2.0,145.405301,CVX,5684
2,DUA408233,"Stock(conId=1520593, symbol='JPM', exchange='N...",2.0,211.180078,JPM,1520593
3,DUA408233,"Stock(conId=8894, symbol='KO', exchange='NYSE'...",-2.0,71.153415,KO,8894
4,DUA408233,"Stock(conId=49462172, symbol='V', exchange='NY...",-2.0,274.480704,V,49462172
5,DUA408233,"Stock(conId=272093, symbol='MSFT', exchange='N...",-8.0,427.848868,MSFT,272093


In [141]:
target

,symbol,position
0,AAPL,1.0
1,MSFT,1.0
2,JPM,1.0
3,V,-1.0
4,CVX,-1.0
5,KO,-1.0


In [143]:
ib.disconnect()